In [4]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from random import randrange
from scipy.optimize import *
import matplotlib.pyplot as plt

Cuda is not available!
logicqubit version 1.5.8


In [5]:
gates = Gates()

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [6]:
III = ID.kron(ID).kron(ID)
XXX = X.kron(X).kron(X)
YYY = Y.kron(Y).kron(Y)
ZZZ = Z.kron(Z).kron(Z)
ZZZ.get()

array([[ 1,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0, -1]])

In [7]:
#H = III -XXX*3 + YYY*4 + 2*ZZZ
H = III*3 + YYY*4 + ZZZ*2 

min(scipy.linalg.eig(H.get())[0])

(-1.4721359549995796+0j)

In [8]:
def _ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        reg[1].CNOT(reg[0])
        for j in range(n_qubits):
            reg[i].RY(params[j])
            
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_3q(q1, q2, q3, params):
    q1.RY(params[0])
    q2.RY(params[1])
    q3.RY(params[2])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q1.RX(params[3])
    q2.RX(params[4])
    q3.RX(params[5])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q1.RY(params[6])
    q2.RY(params[7])
    q3.RY(params[8])
    q2.CNOT(q1)
    q3.CNOT(q2)

In [9]:
def expectation_3q(params):
    logicQuBit  = LogicQuBit(3)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()

    ansatz_3q(q1,q2,q3,params)
    #ansatz([q1,q2,q3],params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_3q, [0,0,0,0,0,0,0,0,0], method='Nelder-Mead',options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:586: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[k] = func(sim[k])
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:618: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:611: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxe
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:614: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:627: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxc
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:637: ComplexWarning: Casting comp

 final_simplex: (array([[-3.60525077,  1.57080132,  1.18012058, -1.57080266,  0.46680586,
         1.86494343, -1.57079746, -0.1568917 ,  1.57079354],
       [-3.60525079,  1.57078494,  1.18012295, -1.57078875,  0.46682062,
         1.86493827, -1.57079823, -0.15689416,  1.57079446],
       [-3.60522503,  1.57080941,  1.18008126, -1.57081888,  0.46665518,
         1.86495151, -1.57075125, -0.15682623,  1.57078741],
       [-3.60524148,  1.57080066,  1.18011314, -1.5707955 ,  0.46680025,
         1.86493957, -1.57077886, -0.15689171,  1.57077469],
       [-3.60523282,  1.57078693,  1.18013807, -1.57076316,  0.46692041,
         1.86492038, -1.57087961, -0.15693767,  1.57081804],
       [-3.60524394,  1.57078514,  1.18011215, -1.57077375,  0.46679038,
         1.86494007, -1.57084089, -0.15687503,  1.57082346],
       [-3.60524484,  1.57077406,  1.18009212, -1.57084081,  0.46662664,
         1.86495482, -1.57068932, -0.15680621,  1.5707867 ],
       [-3.60523456,  1.57078913,  1.18008963

In [10]:
#000 =  1
#001 = -1
#010 = -1
#011 =  1
#100 = -1
#101 =  1
#110 =  1
#111 = -1

def expectation_value(measurements):
    probabilities = np.array(measurements)
    states = np.array([1,-1,-1,1,-1,1,1,-1])
    expectation = np.sum(states * probabilities)
    return expectation

def sigma_xxx(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    q3.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3])
    result = expectation_value(result)
    return result

def sigma_yyy(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    q3.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3])
    result = expectation_value(result)
    return result

def sigma_zzz(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
          
    result = logicQuBit.Measure([q1,q2,q3])
    result = expectation_value(result)
    return result

def expectation_energy(params):
    xxx =  sigma_xxx(params)
    yyy =  sigma_yyy(params)
    zzz =  sigma_zzz(params)

    result = 3 + 4*yyy + 2*zzz
    return result

In [71]:
#initial_values = [random.random() for _ in range(3)]
minimum = minimize(expectation_energy, [0,0,0,0,0,0,0,0,0], method='Nelder-Mead')
print(minimum)

 final_simplex: (array([[-3.60551795,  1.56899032,  1.22124065, -1.52295114,  0.68953303,
         1.83960401, -1.67633677, -0.25644847,  1.58081892],
       [-3.60551592,  1.56898838,  1.22122658, -1.52297746,  0.68945212,
         1.839611  , -1.67626246, -0.25641571,  1.58079414],
       [-3.60550961,  1.56898733,  1.22124431, -1.5229531 ,  0.6895491 ,
         1.83959777, -1.67633799, -0.25645636,  1.58081858],
       [-3.60551386,  1.56897552,  1.22123858, -1.52297246,  0.68949501,
         1.8396024 , -1.67627537, -0.25643219,  1.58080144],
       [-3.60550473,  1.5689861 ,  1.22123355, -1.52299405,  0.68946617,
         1.83960351, -1.67624804, -0.2564248 ,  1.58078991],
       [-3.60550773,  1.56899082,  1.22122836, -1.52293521,  0.68951552,
         1.839602  , -1.67634086, -0.25644044,  1.58081389],
       [-3.60550306,  1.56898549,  1.2212412 , -1.52297973,  0.68951144,
         1.83959795, -1.67628643, -0.25644326,  1.58080226],
       [-3.60550527,  1.56897816,  1.22124409

In [47]:
def gradient(params, evaluate):
    n_params = params.shape[0]
    shift = pi/2
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [54]:
params = np.random.uniform(-np.pi, np.pi, 9)
last_params = np.zeros(9)

In [55]:
lr = 0.1
err = 1
while err > 1e-15:
    grad = gradient(params, expectation_energy)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

<ipython-input-47-cef20e405c15>:15: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


10.004533820473112
0.5533551137481117
0.233968631579363
0.019268585115735745
0.007704532558825826
0.0003560294140712234
0.0005557234110477368
0.00015020791629657406
9.61793953136669e-05
4.054731199529813e-05
2.000867518616722e-05
8.978795452074802e-06
4.128136904735234e-06
1.8483134366920062e-06
8.273270829195486e-07
3.6651696094791575e-07
1.6172455638008285e-07
7.095660549127912e-08
3.101251246917513e-08
1.350202382122134e-08
5.859819240949804e-09
2.5357088651034587e-09
1.0944750217234398e-09
4.713113233023591e-10
2.025393852633783e-10
8.687534025497712e-11
3.7199521241149114e-11
1.5904533245958417e-11
6.789613316016129e-12
2.895594874985363e-12
1.2331358156814075e-12
5.244138456816927e-13
2.2241097852315758e-13
9.514611321037592e-14
3.933520176246929e-14
1.6997514507011146e-14
7.471800955727303e-15
2.8643754035329036e-15
6.772360450213453e-16


In [56]:
expectation_energy(params)

(-1.472135954999581+0j)